# Import Libraries

In [1]:
import copy
import utils
import torch
import constants
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from env import Env
from agent import Agent
from torchsummary import summary
from torch.distributions import Normal, Categorical

# Initialise Environment

In [2]:
#initialise environment
min_x, max_x =  -0.110 - 0.150,   -0.110 + 0.150
min_y, max_y =   0.560 - 0.125,    0.560 + 0.125
min_z, max_z =               0,              0.4 

workspace_lim = np.asarray([[min_x, max_x], 
                            [min_y, max_y],
                            [min_z, max_z]])

print(f"workspace space: \n{workspace_lim}")

obj_dir = 'objects/blocks/'
N_obj   = 4

env = Env(obj_dir, N_obj, workspace_lim, cluttered_mode=True)

workspace space: 
[[-0.26   0.04 ]
 [ 0.435  0.685]
 [ 0.     0.4  ]]


# Initialise Agent

In [3]:
agent = Agent(env, 
              max_memory_size = 20000, 
              is_debug        = True, 
              N_batch         = 256, 
              N_batch_hld     = 256, 
              lr              = 1e-4, 
              hld_lr          = 1e-3,
              tau             = 0.01,
              tau_hld         = 0.01,
              bc_offline      = True,
              max_step        = 150)

device: cuda
[SUCCESS] initialise environment


KeyboardInterrupt: 

# Gather Demonstration Experience

In [ ]:
agent.gather_guidance_experience()

# Push Clone

In [5]:
# push_exp  = agent.buffer_replay_expert_offline.get_experience_by_action_type(constants.PUSH)
# print(f"N_push_exp: {len(push_exp[0])}")
# push_train_loader, push_test_loader = agent.get_train_test_dataloader( push_exp, is_grasp = False, train_ratio=0.9)

In [6]:
# # agent.push_critic1.load_checkpoint()
# # agent.push_critic2.load_checkpoint()
# # agent.push_critic1_target.load_checkpoint()
# # agent.push_critic2_target.load_checkpoint()
# # agent.push_actor.load_checkpoint()

# agent.behaviour_cloning(push_train_loader, push_test_loader, 
#                         agent.push_critic1, agent.push_critic2, 
#                         agent.push_critic1_target, agent.push_critic2_target, 
#                         agent.push_actor, num_epochs = 1000, is_grasp = False)

# push_exp          = None
# push_train_loader = None
# push_test_loader  = None

# Grasp Clone

In [ ]:
grasp_exp = agent.buffer_replay_expert_offline.get_experience_by_action_type(constants.GRASP)
print(f"N_grasp_exp: {len(grasp_exp[0])}")

In [ ]:
# agent.grasp_critic1.load_checkpoint()
# agent.grasp_critic2.load_checkpoint()
# agent.grasp_critic1_target.load_checkpoint()
# agent.grasp_critic2_target.load_checkpoint()
# agent.grasp_actor.load_checkpoint()

grasp_train_loader, grasp_test_loader = agent.get_train_test_dataloader(grasp_exp, is_grasp = True, train_ratio=0.9)
agent.behaviour_cloning(grasp_train_loader, grasp_test_loader, 
                        agent.grasp_critic1, agent.grasp_critic2, 
                        agent.grasp_critic1_target, agent.grasp_critic2_target, 
                        agent.grasp_actor, num_epochs = 250, is_grasp = True)

grasp_exp          = None
grasp_train_loader = None
grasp_test_loader  = None